## 6.3 Batch Normalization

- 前節では「重みの初期値を適切に設定すれば，各層のアクティベーションの分布は適度な広がりを持ち，学習がスムーズに行える」ということを学んだ
- この節ではそのアクティベーション分布を強制的に調整してみる手法 Batch Normalization を紹介する

### 6.3.1 Batch Normalization のアルゴリズム

- 2015 年に提案された手法
- 派生手法もたくさんあるようです
- コンペなどで優れた結果を残している例でよく見かける（というよりほぼ必ず？）

#### Batch Normalization の利点
- 学習を速く進行させることができる（学習係数を大きくすることができる）
- 初期値にそれほど依存しない（初期値に対してそこまで神経質にならなくてよい）
    - この前あった「フレームワークによって同じネットワーク・初期値でもうまく収束したりしなかったりする」問題がなくなる？
- 過学習を抑制する（Dropout などの必要性を減らす）
    - 逆に今 Dropout って使われているんですかね？あまり見かけない気がします．

#### Batch Normalization のアイデア
- 各層でのアクティベーション分布を，適度な広がりを持つように調整すること

#### Batch Normalization の使い方
- データ分布の正規化を行うレイヤをニューラルネットワークに挿入して使う（図6-16）

#### 定義式

ミニバッチ
$$
B = {x_1, x_2, ..., x_m}
$$
に対して

$$
\mu_B = \frac{1}{m} \sum_{i=1}^{m}{x_i}
$$

$$
\sigma_B^2 = \frac{1}{m} \sum_{i=1}^{m}{\left( x_i - \mu_B \right)^2}
$$

$$
\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
$$

式で行っていることは，
- ミニバッチの入力データ ${x1, x2, .., x_m}$ を，平均0，分散1のデータ ${\hat{x}_1, \hat{x}_2, ..., \hat{x}_m}$ に変換する

こと．

ここからさらに 正規化された ${\hat{x}_i}$ に対して，固有のスケール $\gamma$ とシフト $\beta$ で変換を行う．

$$
y_i = \gamma \hat{x}_i + \beta
$$

$\gamma$, $\beta$ はパラメータ，$\gamma = 1, \beta = 0$ を初期値としてこれまでの全結合層と同様に学習を行う．（$\gamma$, $\beta$ はスカラでない場合もある）

微分の導出が意外と大変なんですね...

## 6.4 正則化

- 機械学習の問題では，過学習がよく問題になる
- 過学習とは，訓練データだけに適応しすぎてしまい，訓練データに含まれない他のデータにはうまく対応できない状態
- 未知のデータに対しても正しく識別できるモデルが理想
- NN は複雑で表現力が高いモデルにすることが簡単だが，その分過学習を抑制するテクニックが重要になる

### 6.4.1 過学習

過学習が起きる主な原因
- パラメータを大量に持ち，表現力の高いモデルであること
- 訓練データが少ないこと

今から実際に起こしてみましょう．

### 6.4.2 Weight decay

- 昔からよく用いられているそうです
- Weight decay: L2 ノルムを正則化項として損失関数に加算する正則化のこと
- L1 ノルム正則化もある
- これらを一般化した Lp ノルム正則化もある

ノルム正則化の効果
- L1 ノルム正則化はパラメータのスパース化
- L2 ノルム正則化はパラメータのレンジを狭くする

実際の正則化項をあわせた式は L1 正則化も含めると以下のようになる．

$$
L_{12}\left(W_i\right) = L\left(W_i\right) + \lambda_1 \sum_{i} \left|W_{i}\right| + \frac{1}{2} \lambda_2 \sum_{i} W^2_{i}
$$

学習の際には正則化したい重みでの微分に加えて，正則化項の微分も加算する．

### 6.4.3 Dropout

- モデルが複雑になってくると Weight Decay だけでは対応が困難になる
- そこで Dropout を用いた過学習抑制を行う

何をやっているのか
- ニューロンをランダムに消去しながら学習する手法
- 一定の確率（これは自分で設定する）で Dropout 層に入ってきた値を伝搬させない処理を行う

Dropout の効果
- 毎回異なるニューロンで推論を行うため，NN の中に冗長性が生まれ，アンサンブル学習と同じような効果が期待できる

実際にその効果を動かしながら見てみましょう．

## 6.5 ハイパーパラメータの検証

NN におけるハイパーパラメータとは
- 層の数
- 各層のニューロンの数
- バッチサイズ
- 学習率
- 正則化係数
- etc.

これらを適切な値に設定しなければ，学習はうまく進まない．
しかし探索するのも一苦労．

代表的な方法
- random search
- grid search
- ランダムに gird search を行う方法
- ベイズ最適化を用いた方法